In [2]:
# https://www.relataly.com/crypto-market-cluster-analysis-using-affinity-propagation-python/8114/

In [3]:
# ! pip install cryptocmd
# ! pip install seaborn

In [4]:
from cryptocmd import CmcScraper
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns
from sklearn import cluster, covariance, manifold
import requests
import json

In [5]:
NUMBER_OF_CRYPTOCURRENCIES = 500
TIMEOUT = 10 # seconds

In [65]:
# get a dictionary of the top 500 coin symbols and names from an API

url = f'https://api.coinmarketcap.com/data-api/v3/cryptocurrency/listing?start=1&limit={NUMBER_OF_CRYPTOCURRENCIES}&sortBy=market_cap&sortType=desc&convert=USD&cryptoType=all&tagType=all&audited=false'
response = requests.get(url, timeout = TIMEOUT)
data = json.loads(response.text)
basicData = pd.DataFrame(data['data']['cryptoCurrencyList'])
basicData

,id,name,symbol,slug,tags,cmcRank,marketPairCount,circulatingSupply,selfReportedCirculatingSupply,totalSupply,maxSupply,isActive,lastUpdated,dateAdded,quotes,isAudited,auditInfoList,platform
0,1,Bitcoin,BTC,bitcoin,"[mineable, pow, sha-256, store-of-value, state...",1,10207,1.936919e+07,0.0,1.936919e+07,2.100000e+07,1,2023-05-10T12:43:00.000Z,2010-07-13T00:00:00.000Z,"[{'name': 'USD', 'price': 27977.619752388575, ...",False,NaN,NaN
1,1027,Ethereum,ETH,ethereum,"[pos, smart-contracts, ethereum-ecosystem, coi...",2,6858,1.203177e+08,0.0,1.203177e+08,NaN,1,2023-05-10T12:43:00.000Z,2015-08-07T00:00:00.000Z,"[{'name': 'USD', 'price': 1872.4301521955003, ...",False,[],NaN
2,825,Tether,USDT,tether,"[payments, stablecoin, asset-backed-stablecoin...",3,53077,8.247941e+10,0.0,8.609064e+10,NaN,1,2023-05-10T12:43:00.000Z,2015-02-25T00:00:00.000Z,"[{'name': 'USD', 'price': 1.0006000026612414, ...",True,"[{'coinId': '825', 'auditor': 'CertiK', 'audit...","{'id': 1, 'name': 'Ethereum', 'symbol': 'ETH',..."
3,1839,BNB,BNB,bnb,"[marketplace, centralized-exchange, payments, ...",4,1348,1.558607e+08,0.0,1.558607e+08,NaN,1,2023-05-10T12:43:00.000Z,2017-07-25T00:00:00.000Z,"[{'name': 'USD', 'price': 314.6964053222597, '...",True,"[{'coinId': '1839', 'auditor': 'CertiK', 'audi...",NaN
4,3408,USD Coin,USDC,usd-coin,"[medium-of-exchange, stablecoin, asset-backed-...",5,12313,3.015916e+10,0.0,3.015916e+10,NaN,1,2023-05-10T12:43:00.000Z,2018-10-08T00:00:00.000Z,"[{'name': 'USD', 'price': 1.0000059122280474, ...",False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,21997,Hourglass,WAIT,hourglass,[],496,5,9.795972e+07,97959724.0,9.795972e+07,1.000000e+08,1,2023-05-10T12:43:00.000Z,2022-09-29T05:26:08.000Z,"[{'name': 'USD', 'price': 0.3190003225986774, ...",False,NaN,NaN
496,1974,Propy,PRO,propy,"[real-estate, escrow, smart-contracts, arringt...",497,17,8.205020e+07,0.0,1.000000e+08,NaN,1,2023-05-10T12:43:00.000Z,2017-09-19T00:00:00.000Z,"[{'name': 'USD', 'price': 0.3788036092213025, ...",False,NaN,"{'id': 1, 'name': 'Ethereum', 'symbol': 'ETH',..."
497,5309,OG Fan Token,OG,og-fan-token,"[sports, fan-token, soccer]",498,33,3.857920e+06,3857920.0,5.000000e+06,NaN,1,2023-05-10T12:43:00.000Z,2020-03-12T00:00:00.000Z,"[{'name': 'USD', 'price': 7.948320014408498, '...",False,NaN,"{'id': 8, 'name': 'Chiliz', 'symbol': 'CHZ', '..."
498,15447,Lyra,LYRA,lyra-finance,"[options, optimism-ecosystem]",499,41,3.676934e+08,65000000.0,1.000000e+09,1.000000e+09,1,2023-05-10T12:43:00.000Z,2021-12-01T07:30:40.000Z,"[{'name': 'USD', 'price': 0.08310962930877046,...",True,"[{'coinId': '15447', 'auditor': 'CertiK', 'aud...",NaN


In [67]:
# filter data

basicData = basicData[
    (basicData['isActive'] == 1) & 
    (basicData['dateAdded'].apply(lambda x: int(x[:4])) < 2021) & 
    (basicData['lastUpdated'].apply(lambda x: int(x[:4])) > 2022) &
    (basicData['tags'].apply(lambda x: 'stablecoin' not in x)) &
    (basicData['marketPairCount'] > 5)
]

basicData

,id,name,symbol,slug,tags,cmcRank,marketPairCount,circulatingSupply,selfReportedCirculatingSupply,totalSupply,maxSupply,isActive,lastUpdated,dateAdded,quotes,isAudited,auditInfoList,platform
0,1,Bitcoin,BTC,bitcoin,"[mineable, pow, sha-256, store-of-value, state...",1,10207,1.936919e+07,0.000000e+00,1.936919e+07,2.100000e+07,1,2023-05-10T12:43:00.000Z,2010-07-13T00:00:00.000Z,"[{'name': 'USD', 'price': 27977.619752388575, ...",False,NaN,NaN
1,1027,Ethereum,ETH,ethereum,"[pos, smart-contracts, ethereum-ecosystem, coi...",2,6858,1.203177e+08,0.000000e+00,1.203177e+08,NaN,1,2023-05-10T12:43:00.000Z,2015-08-07T00:00:00.000Z,"[{'name': 'USD', 'price': 1872.4301521955003, ...",False,[],NaN
3,1839,BNB,BNB,bnb,"[marketplace, centralized-exchange, payments, ...",4,1348,1.558607e+08,0.000000e+00,1.558607e+08,NaN,1,2023-05-10T12:43:00.000Z,2017-07-25T00:00:00.000Z,"[{'name': 'USD', 'price': 314.6964053222597, '...",True,"[{'coinId': '1839', 'auditor': 'CertiK', 'audi...",NaN
5,52,XRP,XRP,xrp,"[medium-of-exchange, enterprise-solutions, arr...",6,942,5.183782e+10,0.000000e+00,9.998897e+10,1.000000e+11,1,2023-05-10T12:43:00.000Z,2013-08-04T00:00:00.000Z,"[{'name': 'USD', 'price': 0.43152634071323176,...",False,NaN,NaN
6,2010,Cardano,ADA,cardano,"[dpos, pos, platform, research, smart-contract...",7,782,3.484195e+10,0.000000e+00,3.576134e+10,4.500000e+10,1,2023-05-10T12:43:00.000Z,2017-10-01T00:00:00.000Z,"[{'name': 'USD', 'price': 0.36451595451795615,...",False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,7229,Gelato,GEL,gelato,"[defi, olympus-pro-ecosystem, mvb]",492,31,1.099515e+08,0.000000e+00,4.206900e+08,4.206900e+08,1,2023-05-10T12:43:00.000Z,2020-09-29T00:00:00.000Z,"[{'name': 'USD', 'price': 0.28766643257533886,...",True,"[{'coinId': '7229', 'auditor': 'CertiK', 'audi...",NaN
493,258,Groestlcoin,GRS,groestlcoin,"[mineable, pow, groestl, medium-of-exchange]",494,29,8.251835e+07,0.000000e+00,8.251835e+07,1.050000e+08,1,2023-05-10T12:43:00.000Z,2014-04-11T00:00:00.000Z,"[{'name': 'USD', 'price': 0.38120001934978925,...",False,NaN,NaN
494,7440,BarnBridge,BOND,barnbridge,"[defi, dao, yield-farming, governance, parafi-...",495,74,7.910262e+06,9.322076e+06,1.000000e+07,1.000000e+07,1,2023-05-10T12:43:00.000Z,2020-10-19T00:00:00.000Z,"[{'name': 'USD', 'price': 3.9740495912265703, ...",True,"[{'coinId': '7440', 'auditor': 'CertiK', 'audi...",NaN
496,1974,Propy,PRO,propy,"[real-estate, escrow, smart-contracts, arringt...",497,17,8.205020e+07,0.000000e+00,1.000000e+08,NaN,1,2023-05-10T12:43:00.000Z,2017-09-19T00:00:00.000Z,"[{'name': 'USD', 'price': 0.3788036092213025, ...",False,NaN,"{'id': 1, 'name': 'Ethereum', 'symbol': 'ETH',..."


In [ ]:
basicData.to_csv('./data/basic_data.csv')